In [2]:
import torch
from torchvision import datasets, transforms

In [6]:
dataset_path = "../Dataset"
BATCH_SIZE = 64

In [7]:
transform = transforms.Compose([transforms.Resize((50,50)),
                                transforms.ToTensor()])
dataset = datasets.ImageFolder(f'{dataset_path}/Train', transform=transform)

Dataset ImageFolder
    Number of datapoints: 15089
    Root location: ../Dataset/Train
    StandardTransform
Transform: Compose(
               Resize(size=(50, 50), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )